# MicrostructureNoise

(Available as Jupyter notebook at https://github.com/mschauer/MicrostructureNoise.jl/blob/master/example/MicrostructureNoise.ipynb)

`MicrostructureNoise` is a Julia package for Bayesian volatility estimation in presence of market microstructure noise implementing the methodology described in our new preprint: 

* Shota Gugushvili, Frank van der Meulen, Moritz Schauer, and Peter Spreij: Nonparametric Bayesian volatility learning under microstructure noise.

This blogpost 
gives a short introduction.


## Description

MicrostructureNoise estimates the volatility function $s$ of the stochastic differential equation

$dX_t = b(t,X_t) dt + s(t) dW_t, \quad X_0 = x_0, \quad t \in [0,T]$

from noisy observations of its solution

$Y_i = X(t_i) + V_i, \quad 0 < t_1 < \ldots < t_n = T$

where $\{V_i\}$ denote unobservable stochastic disturbances. The method is minimalistic in its assumptions on the volatility function, which in particular can itself be a stochastic process.

The estimation methodology is intuitive to understand, given that its ingredients are well-known statistical techniques. The posterior inference is performed via the Gibbs sampler, with the Forward Filtering Backward Simulation algorithm used to reconstruct unobservable states $X(t_i)$. This relies on the Kalman filter. The unknown squared volatility function is a priori modelled as piecewise constant and is assigned the inverse Gamma Markov chain prior, which induces smoothing among adjacent pieces of the function. Next to yielding point estimates (e.g. the posterior mean), the method conducts automatic uncertainty quantification via marginal credible bands.


## Setup

Install `MicrostructureNoise` via the package manager. Then load the packages below.
```
Pkg.add("MicrostructureNoise")
```

In [104]:
using MicrostructureNoise, Distributions, Plots
srand(12);

# Real data example

As a first example, we apply our methodology to infer volatility of the high frequency foreign exchange rate data made available by Pepperstone Limited, the London based forex broker (https://pepperstone.com/uk/client-resources/historical-tick-data). Specifically, we use the EUR/USD tick data (bid prices) for 2 March 2015. We retrieve, log-transform and subsample the data and express time in milliseconds.

In [105]:
# uncomment if you do not mind to create this large file 
# Base.download("https://www.truefx.com/dev/data//2015/MARCH-2015/EURUSD-2015-03.zip","./data/EURUSD-2015-03.zip")
# run(`unzip ./data/EURUSD-2015-03.zip -d ./data`)
#dat = readcsv("../data/EURUSD-2015-03.csv")
times = map(a -> DateTime(a, "yyyymmdd H:M:S.s"), dat[1:10:130260,2])
t = Float64[1/1000*(times[i].instant.periods.value-times[1].instant.periods.value) for i in 1:length(times)]
n = length(t)-1
T = t[end]
y = log.(Float64.(dat[1:10:130260, 3]));

In [106]:
plot(t[1:10:end], y[1:10:end], label="EUR/USD")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 0.110 
 
 
 0.111 
 
 
 0.112 
 
 
 0.113 
 
 
 0.114 
 
 
 0.115 
 
 
 0.116 
 
<polyline clip-path="url(#clip3202)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.8557,375.869 45.9762,354.653 46.4369,333.908 46.7297,336.212 46.8917,344.971 47.1745,333.908 47.3467,334.83 47.806,344.51 48.0812,340.822 48.4533,343.127 
 48.6438,340.822 49.1772,349.581 49.5812,343.127 50.4823,335.751 50.8095,344.971 50.9786,343.127 51.1689,337.595 51.3151,346.815 51.7941,361.57 52.0105,360.187 
 52.3095,362.032 52.9262,352.809 53.3316,367.566 53.715,365.721 54.3613,366.182 55.7689,366.182 56.8981,360.648 58.0343,351.887 58.7982,332.525 59.4814,343.127 
 60.4256,355.114 61.1027,355.114 61.9443,352.809 63.1,346.815 63.6548,356.498 64.7365,352.348 65.0937,339.9 66.0371,347.276 66.3109,347.737 66.6903,352.809 
 68.1995,344.049 69.1201,344.049 69.7394,331.142 70.3591,328.838 70.9267,319.16 71.2892,311.327 71.6137,326.073 72.1858,329.76 72.7589,326.073 73.8854,323.768 
 74.7302,315.474 75.1826,318.239 75.9258,322.847 76.6188,311.327 77.1177,316.856 77.4161,327.455 77.5983,334.369 77.9807,328.838 78.7574,337.595 79.376,344.049 
 80.5826,337.595 81.932,337.595 83.077,338.056 83.717,334.369 84.8826,330.22 85.469,320.082 85.6548,323.768 86.3448,326.994 87.2122,322.386 87.803,333.908 
 88.2053,321.464 88.7245,311.788 90.0007,324.69 93.4238,317.317 94.046,310.406 94.9035,314.553 95.7276,310.867 96.4397,313.17 96.7706,311.788 97.0047,315.013 
 97.216,309.485 97.591,304.878 97.9139,305.338 98.4131,311.788 98.7124,296.587 98.9671,301.193 99.4226,304.417 99.8987,312.71 100.586,309.945 101.501,303.035 
 102.078,298.429 103.318,302.575 103.907,315.013 104.361,320.543 105.395,319.16 106.102,320.082 107.234,316.856 108.196,319.621 109.555,323.768 111.405,324.229 
 112.602,321.925 113.033,326.533 113.918,331.603 115.379,321.925 117.626,322.847 118.908,310.867 119.773,305.338 121.232,313.17 121.769,315.935 122.337,319.621 
 123.061,312.71 123.559,303.496 124.353,309.945 125.212,315.935 126.099,307.181 126.57,291.06 127.312,305.799 128.659,297.508 130.074,309.485 131.086,302.114 
 132.373,303.035 133.543,302.114 135.372,303.956 137.507,299.811 139.921,304.878 141.622,302.114 144.592,300.732 147.367,302.114 148.237,292.441 149.012,288.757 
 149.686,296.126 150.807,297.508 151.822,292.902 152.398,291.06 152.916,292.902 153.216,292.441 154.287,292.902 155.787,289.218 156.577,292.441 157.893,292.441 
 160.467,293.363 161.429,296.587 162.904,284.152 163.446,283.231 164.6,289.678 167.412,292.902 169.244,287.376 172.197,275.864 174.118,277.706 175.223,275.404 
 176.563,280.008 178.837,280.008 179.242,269.879 180.159,269.879 180.539,272.642 182.505,274.023 183.478,277.706 184.796,272.642 186.357,271.26 188.184,283.692 
 189.241,282.31 189.966,288.297 191.228,285.994 192.87,280.468 194.067,286.915 195.076,291.981 195.55,288.297 197.773,292.441 199.162,290.599 199.755,296.587 
 200.656,295.205 201.035,304.417 201.204,301.653 201.458,302.114 201.716,313.17 201.857,306.72 201.95,306.72 202.072,307.642 202.5,312.249 203.467,321.003 
 203.964,311.788 204.484,320.543 205.173,327.455 205.607,332.525 206.383,328.377 206.7,316.856 207.412,318.239 208.011,305.338 208.886,310.867 209.472,304.878 
 210.075,307.181 210.803,311.327 212.32,315.474 212.997,315.013 213.682,321.464 214.381,315.935 214.806,321.925 215.166,330.22 215.774,319.621 216.119,311.788 
 216.529,314.553 216.79,310.867 217.144,317.778 217.565,315.013 218.113,319.621 218.699,326.073 219.092,326.073 219.364,339.439 219.586,337.595 219.864,333.447 
 220.217,344.51 220.904,333.908 221.145,323.308 221.436,332.525 221.842,330.22 222.324,316.856 222.625,311.327 223.216,314.092 223.388,303.956 223.801,292.902 
 224.113,297.047 224.346,289.678 224.662,295.665 224.838,293.363 225.22,297.508 225.343,292.902 225.515,2

The prior specification is done via the `Prior` struct.
Most importantly, we set the number of bins `N`.

In [107]:
prior = MicrostructureNoise.Prior(
N = 40,

α1 = 0.0,
β1 = 0.0,

αη = 0.01, 
βη = 0.01,

Πα = LogNormal(1., 0.5),
μ0 = 0.0,
C0 = 5.0
);

Sample the posterior using MCMC.

In [108]:
α = 0.3 # Initial smoothness hyperparameter guess
σα = 0.1 # Random walk step size for smoothness hyperparameter

td, θs, ηs, αs, p = MicrostructureNoise.MCMC(prior, t, y, α, σα, 1500, printiter=500);

500 	 α 5.380198810926286✓	 √η0.001265256088043003
1000 	 α 3.16338362080358✓	 √η0.0012521150797968476
1500 	 α 3.7936135928197197✓	 √η0.0012511490349212352


The inline help gives a synopsis of the function `MCMC`.

In [110]:
?MicrostructureNoise.MCMC

```
MCMC(Π::Union{Prior,Dict}, tt, yy, α0::Float64, σα, iterations; subinds = 1:1:iterations, η0::Float64 = 0.0, printiter = 100) -> td, θ, ηs, αs, pacc
```

Runs the Markov Chain Monte Carlo procedure for `iterations` iterations, on data `(tt, yy)`, where `tt` are observation times and `yy` are observations, `α0` is the initial guess for the smoothing parameter `α` (necessary), `η0` is the initial guess woth the noise variance (optional), and `σα` is the stepsize for the random walk proposal for `α`.

Prints verbose output every `printiter` iteration.

Returns `td, θs, ηs, αs, pacc`. `td` is the time grid of the bins, `ηs`, `αs` are vectors of iterates, possibly subsampled at indices `subinds`, `θs` is a matrix with iterates of `θ` rows, `paccα` is the acceptance probability for the update step of `α`.

Keyword args `fixalpha`, `fixeta` when set to `true` allow fixing `α` and `η` at their initial values.


In [111]:
posterior = MicrostructureNoise.posterior_volatility(td, θs)

tt, mm = MicrostructureNoise.piecewise(posterior.post_t, posterior.post_median[:])

plot(tt, mm, label="posterior median", color="dark blue")
plot!(MicrostructureNoise.piecewise(posterior.post_t, posterior.post_qlow[:])...,
    fillrange = MicrostructureNoise.piecewise(posterior.post_t, posterior.post_qup[:])[2], 
    fillalpha = 0.2,
    alpha = 0.0,
    fillcolor="blue",
    title="Volatility estimate", label="marginal $(round(Int,posterior.qu*100))% credibility band")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 0.0000000002 
 
 
 0.0000000004 
 
 
 0.0000000006 
 
 
 0.0000000008 
 
 
 0.0000000010 
 
 
 Volatility estimate 
 
<polyline clip-path="url(#clip3402)" style="stroke:#00008b; stroke-width:1; stroke-opacity:1; fill:none" points="
 92.6898,373.037 108.005,373.037 108.005,371.12 128.784,371.12 128.784,369.99 149.898,369.99 149.898,368.25 187.297,368.25 187.297,364.282 226.277,364.282 
 226.277,353.776 246.246,353.776 246.246,339.358 256.878,339.358 256.878,340.283 263.899,340.283 263.899,324.381 270.374,324.381 270.374,338.337 277.122,338.337 
 277.122,322.262 283.911,322.262 283.911,275.651 289.445,275.651 289.445,318.407 294.544,318.407 294.544,341.251 299.809,341.251 299.809,353.444 309.678,353.444 
 309.678,353.049 317.96,353.049 317.96,336.922 325.737,336.922 325.737,344.013 331.773,344.013 331.773,333.599 340.948,333.599 340.948,315.532 347.884,315.532 
 347.884,315.514 356.227,315.514 356.227,324.808 366.97,324.808 366.97,299.393 373.415,299.393 373.415,313.212 380.263,313.212 380.263,338.942 387.997,338.942 
 387.997,306.822 393.846,306.822 393.846,239.429 397.772,239.429 397.772,227.266 401.437,227.266 401.437,255.66 405.465,255.66 405.465,306.706 411.654,306.706 
 411.654,324.016 418.021,324.016 418.021,307.267 426.057,307.267 426.057,324.084 434.844,324.084 434.844,334.184 445.327,334.184 445.327,347.888 463.062,347.888 
 463.062,350.097 479.622,350.097 479.622,350.55 497.336,350.55 497.336,362.618 519.182,362.618 519.182,369.137 539.538,369.137 539.538,371.897 580.315,371.897 
 
 "/>
<polygon clip-path="url(#clip3402)" points="
92.6898,375.869 108.005,375.869 108.005,374.273 128.784,374.273 128.784,374.288 149.898,374.288 149.898,374.196 187.297,374.196 187.297,371.379 226.277,371.379 
 226.277,363.747 246.246,363.747 246.246,363.408 256.878,363.408 256.878,359.838 263.899,359.838 263.899,351.131 270.374,351.131 270.374,360.96 277.122,360.96 
 277.122,349.169 283.911,349.169 283.911,314.659 289.445,314.659 289.445,349.433 294.544,349.433 294.544,360.018 299.809,360.018 299.809,366.172 309.678,366.172 
 309.678,367.212 317.96,367.212 317.96,353.548 325.737,353.548 325.737,363.635 331.773,363.635 331.773,356.025 340.948,356.025 340.948,342.111 347.884,342.111 
 347.884,344.651 356.227,344.651 356.227,357.317 366.97,357.317 366.97,334.9 373.415,334.9 373.415,347.217 380.263,347.217 380.263,358.706 387.997,358.706 
 387.997,346.958 393.846,346.958 393.846,300.685 397.772,300.685 397.772,297.021 401.437,297.021 401.437,315.102 405.465,315.102 405.465,341.294 411.654,341.294 
 411.654,354.545 418.021,354.545 418.021,338.588 426.057,338.588 426.057,348.804 434.844,348.804 434.844,351.919 445.327,351.919 445.327,370.148 463.062,370.148 
 463.062,365.257 479.622,365.257 479.622,369.108 497.336,369.108 497.336,370.475 519.182,370.475 519.182,374.592 539.538,374.592 539.538,375.221 580.315,375.221 
 580.315,342.825 539.538,342.825 539.538,356.096 519.182,356.096 519.182,341.075 497.336,341.075 497.336,307.398 479.622,307.398 479.622,318.211 463.062,318.211 
 463.062,311.573 445.327,311.573 445.327,277.077 434.844,277.077 434.844,254.801 426.057,254.801 426.057,204.498 418.021,204.498 418.021,229.72 411.654,229.72 
 411.654,191.004 405.465,191.004 405.465,31.4961 401.437,31.4961 401.437,82.9819 397.772,82.9819 397.772,40.6544 393.846,40.6544 393.846,246.439 387.997,246.439 
 387.997,248.999 380.263,248.999 380.263,215.735 373.415,215.735 373.415,220.781 366.97,220.781 366.97,264.02 356.227,264.02 356.227,251.212 347.884,251.212 
 347.884,194.295 340.948,194.295 340.948,289.517 331.773,289.517 331.773,312.929 325.737,312.929 325.737,304.846 317.96,304.846 317.96,324.654 309.678,324.654 
 309.678,313.226 299.809,313.226 299.809,307.434 294.544,307.434 294.544,248.247 289.445,248.247 289.445,181.942 283.911,181.942 283.911,243.828 277.122,243.828 
 277.12

A histogram of `sqrt.(ηs)` visualises the posterior of the observation error `η`, indicating that there is indeed a non-negligible microstructure noise.

In [112]:
histogram(sqrt.(ηs[end÷2:end]), nbins=20, label="eta")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00124 
 
 
 0.00125 
 
 
 0.00126 
 
 
 0.00127 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 eta

# How good does it work: Test with the Heston model

In [113]:
using Bridge
using StaticArrays
const R2 = SVector{2,Float64};

In [114]:
struct Heston <: ContinuousTimeProcess{R2}
    mu::Float64
    kappa::Float64
    theta::Float64
    sigma::Float64
end
Heston(;mu = NaN, kappa = NaN, theta = NaN, sigma = NaN) = Heston(mu, kappa, theta, sigma)
Bridge.b(s, x, P::Heston) = R2(P.mu*x[1], P.kappa*(P.theta - x[2]))
Bridge.σ(s, x, P::Heston) = Diagonal(R2(sqrt(x[2])*x[1], P.sigma*sqrt(x[2])))

In [115]:
T = 1.0
n = 4000
nf = 10
P = Heston(mu = 0.02, kappa = 7.0, theta = 0.04, sigma = 0.6)

Heston(0.02, 7.0, 0.04, 0.6)

In [116]:
tfine0 = T .* sort(rand(n*nf-1))
tfine = [0.0; tfine0; T]

is = [i <= n-1 for i in 1:length(tfine)-2]
is = [true; is[randperm(length(is))]; true]
t = tfine[is];
    

In [117]:
u = R2(1., P.theta)
W = sample(tfine, Wiener{R2}())
ρ = -0.6
map!(v->R2(v[1], ρ*v[1] + sqrt(1-ρ^2)v[2]), W.yy, W.yy)

Xfine = solve(EulerMaruyama(), u, W, P)

xtrue = log.(first.(Xfine.yy[is]))
s0(t) = sqrt(Xfine.yy[searchsortedfirst(Xfine.tt, t)][2])
y = copy(xtrue) 

η0 = 0.005^2
y .+= randn(n + 1) * sqrt(η0);


In [118]:
α = 5.0 # Initial smoothness hyperparameter guess
σα = 3.0 # Random walk step size for smoothness hyperparameter

td, θs, ηs, αs, p = MicrostructureNoise.MCMC(prior, t, y, α, σα, 10000, printiter=500);

println("acceptance probability $p")

500 	 α 6.853859145847748✓	 √η0.005771926854201921
1000 	 α 2.3675920707901756	 √η0.005699848296846304
1500 	 α 5.192703331810684✓	 √η0.0056889793178693405
2000 	 α 6.672726750385639✓	 √η0.005739263457748688
2500 	 α 10.954010592981682	 √η0.00557738229744995
3000 	 α 7.335157644685448	 √η0.005621248336034742
3500 	 α 3.2354172026801753✓	 √η0.005727353421158912
4000 	 α 2.6568201471608495	 √η0.005689253410411969
4500 	 α 3.099298872537003✓	 √η0.005632507527647822
5000 	 α 7.762047671683255	 √η0.005663289695359922
5500 	 α 5.852303831066731✓	 √η0.005673097727805116
6000 	 α 2.5595148075010314	 √η0.005753480671481345
6500 	 α 7.32620642995769	 √η0.005607045117888518
7000 	 α 2.7717265420538233	 √η0.005898376055226177
7500 	 α 10.030202608593427	 √η0.0057819434558757715
8000 	 α 4.106323352786911	 √η0.005709326208812153
8500 	 α 4.890730550576268✓	 √η0.005731542876067114
9000 	 α 10.340522219236512	 √η0.0056670123045837574
9500 	 α 8.682764529222913✓	 √η0.00564884996370438
10000 	 α 7.6809

In [119]:
posterior = MicrostructureNoise.posterior_volatility(td, θs)

tt, mm = MicrostructureNoise.piecewise(posterior.post_t, posterior.post_median[:])

plot(tt, mm, label="posterior median", linewidth=0.5, color="dark blue")
plot!(t[1:10:end], (s0.(t[1:10:end])).^2, label="true volatility", color="red")


plot!(MicrostructureNoise.piecewise(posterior.post_t, posterior.post_qlow[:])...,
    fillrange = MicrostructureNoise.piecewise(posterior.post_t, posterior.post_qup[:])[2], 
    fillalpha = 0.2,
    alpha = 0.0,
    fillcolor="darkblue",
    title="Volatility estimate", label="marginal $(round(Int,posterior.qu*100))% credibility band")



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Volatility estimate 
 
<polyline clip-path="url(#clip3802)" style="stroke:#00008b; stroke-width:0.5; stroke-opacity:1; fill:none" points="
 39.1651,290.93 54.0487,290.93 54.0487,266.689 66.8451,266.689 66.8451,196.187 80.4485,196.187 80.4485,192.671 92.8147,192.671 92.8147,151.864 106.874,151.864 
 106.874,246.248 121.258,246.248 121.258,321.236 134.136,321.236 134.136,344.01 149.621,344.01 149.621,331.675 163.131,331.675 163.131,324.008 179.147,324.008 
 179.147,317.239 191.974,317.239 191.974,317.357 205.759,317.357 205.759,329.567 221.786,329.567 221.786,347.875 234.321,347.875 234.321,359.539 247.712,359.539 
 247.712,362.258 260.304,362.258 260.304,362.612 272.361,362.612 272.361,347.204 286.726,347.204 286.726,357.92 298.84,357.92 298.84,356.286 312.303,356.286 
 312.303,347.969 326.234,347.969 326.234,323.659 340.794,323.659 340.794,320.173 354.53,320.173 354.53,308.494 366.113,308.494 366.113,289.015 379.848,289.015 
 379.848,233.7 394.299,233.7 394.299,250.336 407.559,250.336 407.559,277.354 420.336,277.354 420.336,289.551 435.127,289.551 435.127,310.079 447.885,310.079 
 447.885,266.619 462.826,266.619 462.826,249.864 474.769,249.864 474.769,277.349 487.684,277.349 487.684,302.2 501.511,302.2 501.511,263.862 515.823,263.862 
 515.823,290.489 529.731,290.489 529.731,346.929 542.913,346.929 542.913,354.852 552.863,354.852 552.863,342.176 566.449,342.176 566.449,308.133 580.315,308.133 
 
 "/>
<polyline clip-path="url(#clip3802)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1651,215.913 40.2139,252.902 42.2535,243.329 44.0549,254.462 45.1143,209.253 47.1001,260.344 48.569,297.493 49.7611,295.749 51.0044,278.411 52.9086,312.012 
 54.0487,310.381 55.3572,281.152 56.2358,262.657 57.3926,279.062 58.8919,252.916 60.0926,226.723 61.7992,266.911 62.7367,271.937 64.0282,281.734 65.6265,286.268 
 66.8451,293.733 68.4325,257.628 69.7553,265.665 71.0192,246.033 72.8564,215.085 73.815,222.281 75.5045,215.461 77.2732,174.006 78.2298,200.072 79.0937,190.678 
 80.4485,190.19 81.9014,186.765 82.8507,174.353 84.124,164.496 86.0196,149.467 87.3465,131.757 89.293,160.561 89.9949,176.311 91.2808,180.715 92.0178,177.736 
 92.8147,195.583 94.1072,180.826 96.1252,242.869 97.0725,245.767 98.6991,215.487 100.24,218.385 101.592,204.766 102.568,159.455 104.619,124.322 105.359,103.736 
 106.874,106.313 107.933,116.659 109.332,114.972 110.797,134.232 112.371,178.988 114.555,184.113 116.52,177.445 117.832,255.803 119.227,283.261 120.411,286.552 
 121.258,312.81 122.748,309.463 123.443,307.819 124.582,327.38 126.476,310.096 127.923,315.014 128.654,284.266 130.803,307.698 132.154,321.819 133.387,322.206 
 134.136,322.539 136.073,348.192 138.149,347.964 140.453,337.906 142.141,336.724 143.293,314.951 144.318,318.018 145.48,322.38 146.976,313.473 148.561,301.7 
 149.621,291.735 150.981,306.789 152.356,313.483 153.692,302.104 154.844,291.547 155.934,294.493 157.346,284.761 159.172,273.52 160.586,281.854 162.09,284.813 
 163.131,277.804 164.582,288.066 165.798,287.799 167.293,304.247 169.067,304.523 170.742,302.481 172.631,295.238 173.946,268.899 175.853,274.958 177.059,235.547 
 179.147,241.448 180.561,293.658 181.235,285.862 182.805,311.977 183.912,305.708 185.664,315.817 187.675,318.729 189.171,296.354 190.224,297.565 190.863,281.031 
 191.974,282.749 192.952,293.52 193.831,273.905 195.292,255.248 196.341,240.606 198.595,224.034 201.553,281.404 202.678,283.694 203.518,277.833 205.023,242.205 
 205.759,242.838 207.384,247.422 208.856,236.691 211.056,251.906 212.596,275.396 213.982,285.039 215.388,284.111 216.923,275.058 218.452,288.451 219.425,336.817 
 221.786,333.889 223.096,336.403 223.99,351 225.075,357.416 226.386,358.965 227.284,364.322 228.872,355.038 230.332,349.741 232.322,336.266 233.35

In [120]:
histogram(sqrt.(ηs[end÷2:end]), nbins=20, label="eta")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0055 
 
 
 0.0056 
 
 
 0.0057 
 
 
 0.0058 
 
 
 0.0059 
 
 
 0.0060 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 1200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 eta

## References

* Shota Gugushvili, Frank van der Meulen, Moritz Schauer, and Peter Spreij: Nonparametric Bayesian volatility estimation. [arxiv:1801.09956](https://arxiv.org/abs/1801.09956), 2018.

* Shota Gugushvili, Frank van der Meulen, Moritz Schauer, and Peter Spreij: Nonparametric Bayesian volatility learning under microstructure noise. In preparation.